# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [1]:
!pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math

Defaulting to user installation because normal site-packages is not writeable


**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [2]:
nlen = 2048

def find_p_q():
  p = cu.getStrongPrime(1024)
  q = cu.getStrongPrime(1024)
  while p > pow(2, nlen//2) or p < pow(2, (nlen - 1)//2) or q > pow(2, nlen//2)\
         or q < pow(2, (nlen - 1)//2) or abs(p - q) < pow(2, (nlen//2) - 100):
    p = cu.getStrongPrime(1024)
    q = cu.getStrongPrime(1024)
  return p, q

def generate_keypair():
  p, q = find_p_q()
  n = p * q
  print("nlen:", len(bin(n))-2)
  f = (p-1) * (q-1)
  e = cu.getRandomRange(65537, min(pow(2, 256), f))

  while math.gcd(e, f) != 1 or e > pow(2, 256) or math.lcm(p-1, q-1) < e*pow(2, nlen//2):
    e = cu.getRandomRange(65537, min(pow(2, 256), f))

  d = pow(e, -1, f)
  return n, e, d

n, e, d = generate_keypair()


nlen: 2048


## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [3]:
M = "Ala ma kota"

# Your code here

# encryption
M_num = [ord(c) for c in M]
print(M_num)
C = [pow(m, e, n) for m in M_num]
print(C)



[65, 108, 97, 32, 109, 97, 32, 107, 111, 116, 97]
[5837181387046673482521599191049587570511958899281609443816826789565066011624309681289717811693693090046472073529947936675300837265267855330119035122456003703536458726638843438321398459094884283583093418328919430142129551814471890589710755943407257534285983402393150670156924772355461792744168658393241483893253926642299397204155593234263881522452398163326358463395296689776639541564715442173764561369127650976506214964102125530149484014197867518292166134902285134251170115804827058410562785023969662162862350719593243011047433551248074863704115160752667477305997997231190737330709616619491856149346282414276611566761, 8844979722918885595802498549731003841456496427779725603726244742973132125766952597493097395251914050245186693849049597376513229061808728720310608225368330942105107540220947401578047560432664716983855147137268727852598398068531351968213260589399061129855002430251658752120114035841312602006825560943168441204936188949803493572033036

In [4]:
def encode_str(s):
  s_num = ''.join([hex(ord(c))[2:].zfill(2) for c in s])
  return int(s_num, 16)

# encryption

def encrypt(e, n, mes):
  m_num = encode_str(mes)
  print('message number ', m_num)
  c = pow(m_num, e, n)
  return c


# decryption

def decrypt(d, n, c):
  m_decrypted = pow(c, d, n)
  print('decrypted message number ', m_decrypted)
  message = ''
  m_dec = m_decrypted
  while m_dec:
    m = m_dec & 0xFF
    message = chr(m) + message
    m_dec = m_dec >> 8
  return message


M = "Ala ma kota"

C = encrypt(e, n, M)
print('ciphertext: ', C)

message = decrypt(d, n, C)
print('decrypted message ', message)

message number  79091985525906541409956961
ciphertext:  20273077197344753406198085961640242152473827810408773002530386173236232056260397578377452702715023390653053980703955420516401951331954790900151724958669332362219290543964753724202802413988468465909685525759952458325212125201542478500244231035087979795802067423970850470530339972322695645838772420456348813961432879704525280925507501824572722825130148305509671183695053365352141502459182983921799953273139699002558348171664804101054583577418930954511018708577231637700302133469425661417517814545278570989834134457737961445964091049620081963370837204632080340306275982760941525116538994463293251376361908062170813965716
decrypted message number  79091985525906541409956961
decrypted message  Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [5]:
M = "Ala ma kota"

hash_object = hl.sha256(M.encode())
h = hash_object.hexdigest()
h = encode_str(h)
SIG = pow(h, d, n)
print(SIG)

VER = pow(SIG, e, n)
print()
print(VER)
print(h)
print(VER == h)

2706781031126678406837875379251672446692325473115390862049493482649841565366035686600306786383729539119692080009710710740124780741057128188916432604192128637673281790982375691441861475139743267917807417520627434983059625284323841545996468504133086230696280421481355449197980562665742867163570405526130367704989202817205314568416244757780920112511644739571442226563620046172492282642166992241034631732379158289286931084748183513865638731274063963982504986259942697327469748029861778919358376132134321278323107760557438011928108662732463247436717545252291339464332935433520792812450917679276880175677998160404397353873

2576609446136810321284793642849579379527454191856329747264527660897419943862306273701113910862743553954351156540019741177436361715134805108682805972579941
2576609446136810321284793642849579379527454191856329747264527660897419943862306273701113910862743553954351156540019741177436361715134805108682805972579941
True


## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [6]:
def mgf1(M, length):
  hlen = 256 // 8
  iter = math.ceil(length / hlen)
  output = bytes()

  for i in range(iter):
    tmp = M + i.to_bytes(4, byteorder='big')
    h = hl.sha256(tmp).digest()
    output = output + h
  return output[:length]

M = "Ala ma kota"
print(mgf1(M.encode(), 128))

b'O\x82y,\xd5\x98\x18\xc5B7\xc4YT\xe6\xfa\x82\xdd\x08\xc8\x8d\x9b?\xb2\xe2X3\x1c\tf%d\x18\xd6,\xad\x91\x8eH\x9e\xa6\x1b\xf7m\xd4\xfcu\xf5L\x0f&\x94\x10\x1a\x89\xf0e\xcdx\xb3\xa3?\x7f\xca\xbd.M#C"\xb4B\x8d\xd3\xb1GB\xe3y5\xc9 \xd4\t`B\xfa\xa9\x9e\x0c#\xea4Hb\xe0\x02\xbd\xdeG\xd2\x15\xfd\xff\xc7f|\x18~\x84{B\xa8O\xa6\xe1\xd4\x9e=\x1d\xf5Q65$\xd2\xeeE#'


## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [8]:
import random

M = "Ala ma kota"

# Your code here

def xor_bytes(a, b):
  result = bytes(x ^ y for x, y in zip(a, b))
  return result

def oaep_encoding(k, M):
  hlen = 256 // 8
  mlen = len(M)

  if mlen > k - 2 * hlen - 2:
    print('ERROR')
    return None
  
  lHash = hl.sha256("".encode()).digest()
  PS = b'\x00' * (k - mlen - 2 * hlen - 2)
  DB = lHash + PS + b'\x01' + M

  seed = random.randbytes(hlen)
  dbMask = mgf1(seed, k-hlen-1)
  maskedDB = xor_bytes(dbMask, DB)

  seedMask = mgf1(maskedDB, hlen)
  maskedSeed = xor_bytes(seedMask, seed)

  
  EM = b'\x00' + maskedSeed + maskedDB
  return EM

def oaep_decoding(k, EM, mlen):
  hlen = 256 // 8
  Y = EM[0]
  maskedSeed = EM[1:hlen+1]
  maskedDB = EM[hlen+1:]
  lHash = hl.sha256("".encode()).digest()
  seedMask = mgf1(maskedDB, hlen)
  seed = xor_bytes(seedMask, maskedSeed)
  dbMask = mgf1(seed, k-hlen-1)
  DB = xor_bytes(maskedDB, dbMask)
  lHashPrime = DB[:hlen]
  PS = DB[hlen:hlen + (k - mlen - 2*hlen -2)]

  if Y != 0:
    print('ERROR Y != 0')
    return None

  if DB[hlen + (k - mlen - 2*hlen -2)] != 0x01:
    print("ERROR 0")
    return None
  M = DB[hlen + (k - mlen - 2*hlen -2)+1:]

  if lHashPrime != lHash:
    print('ERROR 1')
    return None
  return M


EM = oaep_encoding(nlen//8, M.encode())
oaep_decoding(len(EM), EM, len(M.encode()))

b'Ala ma kota'

In [9]:
def encrypt_oaep(e, n, mes):
  print(mes)
  m_num = oaep_encoding(nlen//8, mes)
  m_num = int.from_bytes(m_num, byteorder='big')
  print('message number ', m_num)
  c = pow(m_num, e, n)
  return c

def decrypt_oaep(d, n, c, mlen):
  m_decrypted = pow(c, d, n)
  print('decrypted message number ', m_decrypted)
  num_bytes = (m_decrypted.bit_length() + 7) // 8
  print(num_bytes)
  m_decrypted = m_decrypted.to_bytes(nlen//8, byteorder='big')
  message = oaep_decoding(nlen//8, m_decrypted, mlen)
  return message


EM = encrypt_oaep(e, n, M.encode())
print('\n\n')
print(EM)
print('\n\n')
decrypt_oaep(d, n, EM, len(M.encode()))

b'Ala ma kota'
message number  10756041417536351648920865156271312658069187344209125717716679830523513838380580085915895349736996289189101149015464425804960743139562395204474824743487879931854617778588738658063413767203554821975100079796644631964506522227803318205629684455154373129095364210890964567246532260141442334632050081576826296703857094696087903266042958087003152299151605135057131870569028975674661656867642197542049274821011916582046246672691916707993566882016912005061348454737319862660593772180562931510168044339019409443103676822139206947221432186813263552788916432220529532584109259153706547788314675595332656165768267421027223734



157902320502116347093956474602635226733672439890278650136469232693929141639040134586243318186592950085504586336841392776956903094793016039472501793884452594756403673016542437165258211332819876061862499309495022697912003546233214159238428644500176850070752698159393848599592623737555230898381038844618749708709918865277579992181509283082230620403956264

b'Ala ma kota'

## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [10]:
m = "Ala ma podpis"

def xor_bytes(a, b):
  result = bytes(x ^ y for x, y in zip(a, b))
  return result

def zero_most_significant_bits(b, n):
    mask = (1 << n) - 1 
    b_new = bytearray(b)  
    
    for i in range(len(b_new)):
        b_new[i] &= ~mask 
    
        if n > 8:
            mask = 0xFF  
            n -= 8
        else:
            mask >>= n 
    
    return bytes(b_new)


def emsa_encoding(slen, M, emBits):
  emlen = math.ceil(emBits/8)
  hlen = 256 // 8
  mlen = len(M)
  mHash = hl.sha256(M).digest()
  if emlen < hlen + slen + 2:
    print('ERROR')
    return None

  salt = random.randbytes(slen)
  MPrime = b'\x00' * 8 + mHash + salt 

  mHashPrime = hl.sha256(MPrime).digest()
  
  PS = b'\x00' * (emlen - slen - hlen -2)

  DB = PS + b'\x01' + salt

  dbMask = mgf1(mHashPrime, emlen - hlen -1)
  maskedDB = xor_bytes(DB, dbMask)
  maskedDB = zero_most_significant_bits(maskedDB, 8*emlen - emBits)
  EM = maskedDB + mHashPrime + b'\xbc'
  return EM

def emsa_decoding(slen, EM, M, emBits):
  emlen = math.ceil(emBits/8)
  hlen = 256 // 8
  mlen = len(M)
  mHash = hl.sha256(M).digest()
  if emlen < hlen + slen + 2:
    print('ERROR')
    return None
  if hex(EM[-1]) != '0xbc':
    print('ERROR')
    return None
  maskedDB = EM[:(emlen - hlen -1)]
  mHashPrime = EM[(emlen- hlen-1): (emlen - hlen -1) + hlen]
  dbMask = mgf1(mHashPrime, emlen - hlen -1)
  DB = xor_bytes(maskedDB, dbMask)
  DB = zero_most_significant_bits(DB, 8*emlen - emBits)
  for i in range(emlen - hlen - slen - 2):
    if DB[i] != 0:
      print('ERROR 0')
      return None
  if DB[emlen - hlen - slen -2] != 1:
    print('ERROR 1')
    return None
  
  salt = DB[-slen:]
  MPrime = b'\x00' * 8 + mHash + salt
  mHashBis = hl.sha256(MPrime).digest()
  print(mHashPrime)
  print(mHashBis)
  if mHashPrime == mHashBis:
    print("SUCCESS")
  else:
    print("VERIFICATION FAILED")
  return mHashPrime == mHashBis


EM = emsa_encoding(256//8, M.encode(), 2047)

emsa_decoding(256//8, EM, M.encode(), 2047)

b'\xc5!\xdfY\xa2\x81:\xa9dm~1w\xf9\x0f\xa0N5N\x85\xb6A\xf3\xfcFm.\xe7L\x08\xcd%'
b'\xc5!\xdfY\xa2\x81:\xa9dm~1w\xf9\x0f\xa0N5N\x85\xb6A\xf3\xfcFm.\xe7L\x08\xcd%'
SUCCESS


True

In [11]:
M = "Ala ma kota"

hash_object = hl.sha256(M.encode())
h = hash_object.hexdigest()
h = encode_str(h)
SIG = pow(h, d, n)
print(SIG)

VER = pow(SIG, e, n)
print()
print(VER)
print(h)
print(VER == h)

2706781031126678406837875379251672446692325473115390862049493482649841565366035686600306786383729539119692080009710710740124780741057128188916432604192128637673281790982375691441861475139743267917807417520627434983059625284323841545996468504133086230696280421481355449197980562665742867163570405526130367704989202817205314568416244757780920112511644739571442226563620046172492282642166992241034631732379158289286931084748183513865638731274063963982504986259942697327469748029861778919358376132134321278323107760557438011928108662732463247436717545252291339464332935433520792812450917679276880175677998160404397353873

2576609446136810321284793642849579379527454191856329747264527660897419943862306273701113910862743553954351156540019741177436361715134805108682805972579941
2576609446136810321284793642849579379527454191856329747264527660897419943862306273701113910862743553954351156540019741177436361715134805108682805972579941
True


In [12]:
def create_digital_signature(M):
    EM = emsa_encoding(256//8, M, 2047)
    EM_num = int.from_bytes(EM, byteorder='big')
    SIG_num = pow(EM_num, d, n)
    SIG = SIG_num.to_bytes((SIG_num.bit_length() + 7) // 8, byteorder='big')
    return SIG, M

def verify_digital_signature(SIG, M):
    SIG_num = int.from_bytes(SIG, byteorder='big')
    VER_num = pow(SIG_num, e, n)    
    VER = VER_num.to_bytes((VER_num.bit_length() + 7) // 8, byteorder='big')
    return emsa_decoding(256//8, VER, M, 2047) is True
    

SIG, EM = create_digital_signature(M.encode())
verify_digital_signature(SIG,EM)

b'\x8b\x15\x1d<\xc8\xdf\xe3\xd5\x93p\xdf\x9e40\xd71R\xed\x10\xfad\x96\xc0\xa1\x87\xb6~\x18/\x15\x06\xef'
b'\x8b\x15\x1d<\xc8\xdf\xe3\xd5\x93p\xdf\x9e40\xd71R\xed\x10\xfad\x96\xc0\xa1\x87\xb6~\x18/\x15\x06\xef'
SUCCESS


True

## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [13]:
def RSA_KEM_enc(key):
    C = encrypt_oaep(e, n, key)
    C = C.to_bytes((C.bit_length() + 7) // 8, byteorder='big')
    return C

def RSA_KEM_dec(C_key):
    C_key_num = int.from_bytes(C_key, byteorder='big')
    key = decrypt_oaep(d, n, C_key_num, 256//8)
    return key

rand = cu.getRandomNBitInteger(2048)

key = hl.sha256(rand.to_bytes(2048//8, byteorder='big')).digest()
print('KEY: ', key)
C = RSA_KEM_enc(key)
print('\n\nciphered ', C)
decrypted_key = RSA_KEM_dec(C)
print('\n\nDECRYPTED KEY: ', decrypted_key)
print('DEC KEY == KEY: ', decrypted_key == key)

KEY:  b'\x03\x9d\xdb\x91\xc6V\x06\xe7Y"\x01\x99j\x90N\xaf\xfe\x1aPkcY\x93\xd5\x1a\xadU\xac\xfeB#f'
b'\x03\x9d\xdb\x91\xc6V\x06\xe7Y"\x01\x99j\x90N\xaf\xfe\x1aPkcY\x93\xd5\x1a\xadU\xac\xfeB#f'
message number  96730105070306455342587706473516848117813292342586937000421802992337869889190354802912460103830075714428033414078571150445744716321255707634608994700045271930358164337015429764644188242420107068204196817047761701510006950514577751476976567629187240759412753770008925931551982196715226744720234724569121405367202131555769858251975144314829198627486160087289163580460582123183269702192078700882203217789121350543411483723588855236442144047016260597284456569735039097052988884434468070482930001580287430367305651149448075828586679963173558785237692734820367178323241083909122791481722735928793982323457999560034882497


ciphered  b'\xa0\xd9\xd4\xb45\xe3\xd4\x93O\xe0\x94\x8b\r\x07\xd0R\x02\x8f<\x94\xf5\xbfg\xd3\x8d\x98Su!\xe0\xae\xa8\x1e\xbfq\xfb\xcd\x9e\xc6&\xb4\xb2\xee\x91\x0e\xdeA\xd0e6|\x0